**Purpose: To find, genes that overlap with repeats, and find genes that overlap with more than 50% of a repeat 
to be filtered out latter

**Steps: 1. intersect c. elegans repeats with c. elegans genes
         2. calculate % overlap of genes with repeats
         3. note HS-UP genes that overlap with more than 50% of repeat --those will be filtered out in diff notebook

In [109]:
# import libraries 
from __future__ import division 
import pandas as pd
import numpy as np

In [110]:
repeats = pd.read_table('repeats.bed') # from UCSC
genes = pd.read_table('genes.ws245.bed') # from UCSC
genes_not_filtered = pd.read_csv('/home/wschrein/final/Figure1/up_genes_no_intron_filtering.csv') # all up_genes before filtering 

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [111]:
# repeat bed file
repeats.head(1)

,chrI,4194254,4194376,CELE2,738,+
0,chrI,655143,655494,CELE1,1812,+


In [112]:
# use bedtools intersect with -wao feature to find # of bp that overlap

! bedtools intersect -a genes.ws245.bed -b repeats.bed -wao > repeats_intersected_with_genes.txt

In [114]:
# read in interscet file
intersect = pd.read_table('repeats_intersected_with_genes.txt',
      names=['chrom','start','end','gene','score','strand',
             'chrom_r','start_r','end_r','repeat','score_r','strand','bp_overlap'])

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


In [115]:
intersect.head(1) # now find genes that overlap with a repeat by more than 50% 

,chrom,start,end,gene,score,strand,chrom_r,start_r,end_r,repeat,score_r,strand.1,bp_overlap
0,chrI,8377601,8392753,T19A6.1a.2,0,-,chrI,8379433,8379453,(TC)n,16,+,20


In [116]:
# get the length of repeats and genes
intersect['gene_length'] = intersect['end'] - intersect['start']
intersect['repeat_length'] = intersect['end_r'] - intersect['start_r']



In [117]:
# get percent of gene that overlaps with gene 
intersect['percent_overlap'] = (intersect['bp_overlap'] / intersect['gene_length'])*100


In [118]:
# seperate dataframe that contains genes with large overlap
# note: a lot of pseudogenes contain large overlap 

intersect_large_overlap = intersect[intersect.percent_overlap>50]

In [119]:
# make list of genes with more than 50% overlap
genes_with_more_than_50percent = list(intersect_large_overlap['gene'])

In [120]:
# ID up genes that contain more than 50% overlap, 
reapeats_to_be_filtered = []
for index, row in genes_not_filtered.iterrows():
    if row.transcript in genes_with_more_than_50percent:
        print(row.transcript) #  these are the genes that will be filtered out b/c they overlap with more than 50% of a repeat
        reapeats_to_be_filtered.append(row.transcript)



Y53F4B.7
F33H12.6
F59H6.5
Y81B9A.1


In [90]:
# make csv of genes to filter out 

In [121]:
# ID up genes that contain more than 50% overlap, and make 
greater_than_50 = []
for index, row in genes_not_filtered.iterrows():
    if row.transcript in reapeats_to_be_filtered:
        greater_than_50.append("yes")
    else:
        greater_than_50.append("no")
         


In [122]:
genes_not_filtered['greater_than_50'] = greater_than_50

In [123]:
repeats_to_be_filtered = genes_not_filtered[genes_not_filtered.greater_than_50=='yes']

In [124]:
repeats_to_be_filtered=repeats_to_be_filtered[['transcript']]

In [125]:
## This file will be used to filter out genes that overlap with more than 50% of a repetitive read
## This file is used in D1. Filter_the_up_Genes.ipynb

repeats_to_be_filtered.to_csv('repeats_to_be_filtered.csv', index=False)